In [2]:
import pandas as pd
import numpy as np
import folium 
from folium import plugins
import seaborn as sns

In [3]:
df_casos = pd.read_csv(
    '20220220_dados_covid_municipios_sp.csv' , # caminho do arquivo
    encoding= 'iso-8859-1', # encoding, forma que o arquivo foi salvo (caracteres)
    sep = ';' #separados das colunas
)

In [4]:
df_casos['nome_drs'].unique()

array(['Marília', 'São José do Rio Preto', 'São João da Boa Vista',
       'Campinas', 'Bauru', 'Piracicaba', 'Sorocaba',
       'Presidente Prudente', 'Barretos', 'Ribeirão Preto', 'Araçatuba',
       'Araraquara', 'Taubaté', 'Franca', 'Grande São Paulo', 'Registro',
       'Baixada Santista', nan], dtype=object)

In [5]:
df_casos['nome_drs'] = np.where(df_casos['nome_drs'].isnull() == True, 'Sem DRS', df_casos['nome_drs'])

In [6]:
# cria um dataframe em branco com as colunas definidas
df_geo = pd.DataFrame({'CD_MUN': [0], 'NM_MUN': [0], 'SIGLA_UF': [0], 'AREA_KM2': [0], 'geometry': [0]})

# looping para ler todas as linhas do arquivo municipios_sp.json
for idx, info in pd.read_json('municipios_sp.json').iterrows():
    
    # transforma a coluna features em um dicionario
    dict_info = dict(info['features'])
    
    # adiciona as linhas do dicionario no dataframe
    df_geo = df_geo.append({
        'CD_MUN': dict_info['properties']['CD_MUN'],
        'NM_MUN': dict_info['properties']['NM_MUN'],
        'SIGLA_UF': dict_info['properties']['SIGLA_UF'],
        'AREA_KM2': dict_info['properties']['AREA_KM2'],
        'geometry': dict_info['geometry']
        
    } ,ignore_index=True)

In [7]:
df_geo['CD_MUN'] = df_geo['CD_MUN'].astype(int)
df_casos['codigo_ibge'] = df_casos['codigo_ibge'].astype(int)

# Vamos atualizar o df_geo
df_geo = pd.merge(
    df_geo,
    
    #agrupa o df_casos para somar as colunas casos e obitos
    df_casos.groupby(['codigo_ibge', 'nome_drs']).agg({'casos_novos': np.sum, 'obitos_novos': np.sum}).reset_index(),
    
    # une os dois dataframes com base nas informacoes do codigo do ibge 
    how = 'inner',
    left_on = 'CD_MUN',
    right_on = 'codigo_ibge'
)

In [8]:
df_geo.head()

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry,codigo_ibge,nome_drs,casos_novos,obitos_novos
0,3500105,Adamantina,SP,411.987,"{'type': 'Polygon', 'coordinates': [[[-51.0542...",3500105,Marília,6222,159
1,3500204,Adolfo,SP,211.055,"{'type': 'Polygon', 'coordinates': [[[-49.6579...",3500204,São José do Rio Preto,636,22
2,3500303,Aguaí,SP,474.554,"{'type': 'Polygon', 'coordinates': [[[-46.9764...",3500303,São João da Boa Vista,5386,126
3,3500402,Águas da Prata,SP,142.673,"{'type': 'Polygon', 'coordinates': [[[-46.7350...",3500402,São João da Boa Vista,1218,35
4,3500501,Águas de Lindóia,SP,60.126,"{'type': 'Polygon', 'coordinates': [[[-46.6061...",3500501,Campinas,3745,57


In [11]:
def style_3(feature):
    tt = dict(feature['geometry'])
    var = df_geo[df_geo['geometry'] == tt].iloc[0][VAR_METRICA] 
    
    if var <= df_geo[VAR_METRICA].quantile(0.25):
        color_ = '#00b4d8'
    elif var > df_geo[VAR_METRICA].quantile(0.25) and var <= df_geo[VAR_METRICA].quantile(0.50):
        color_ = '#0077b6'
    elif var >= df_geo[VAR_METRICA].quantile(0.50) and var <= df_geo[VAR_METRICA].quantile(0.75):
        color_ = '#023e8a'
    elif var > df_geo[VAR_METRICA].quantile(0.75):
        color_ = '#03045e'

    return {'fillColor': color_,'color': 'white','weight': 1, 'fillOpacity': 0.8} 

    
mapa = folium.Map( location=[-22.2298654,-46.4639913], zoom_start=7, tiles='CartoDB positron')

VAR_METRICA = 'casos_novos'

for nm_drs in df_geo['nome_drs'].unique().tolist():
    casos_ =  folium.FeatureGroup(name = nm_drs).add_to(mapa)
    for idx, info in df_geo[ df_geo['nome_drs'] == nm_drs ].iterrows():
        folium.GeoJson(
            info['geometry'], 
            style_function = style_3,
            tooltip = "{0}: {1} Casos".format(info['NM_MUN'], info['casos_novos'])
        ).add_to(casos_)
    
folium.LayerControl().add_to(mapa)

mapa.save('estado_sao_paulo_casos.html')